# 제조 공정 예측 유지 보수하기

## 분석 프로세스 수립

### 데이터 수집  
데이터 소스: https://www.kaggle.com/datasets/yuansaijie0604/xinjiang-pm?resource=download  

사용되는 데이터 파일 목록
- PdM error
- PdM failure
- PdM machines
- PdM components
- PdM telemetry.

### 데이터 전처리  
각 데이터별 전처리 진행  

Feature 종류
- 일정 시간 통계량 Feature
- 범주형 Feature
- Target Feature

### 모델 생성  
- XGBoost Model 생성
- 모델 평가

### 모델 평가 및 의사결정
- Feature Importance